<a href="https://colab.research.google.com/github/root-epifit/HPC_Sber_2025/blob/main/%D0%94%D0%B72_in_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание 2

### связь Python с программами на C/C++

1. Реализовать на языке C/C++ классические операции перемножения квадратных матриц и умножения матрицы на вектор (15%)

In [ ]:
# можно использовать %%writefile и оформить сишный код прямо здесь
# но наверное удобней будет писать его напрямую в отдельных файлах
# и прислать вместе с домашним заданием

In [39]:
%%writefile matrix_operations.c

#include <stdio.h>
#include <stdlib.h>

#define MAX_SIZE 8192 // Максимальный размер матриц и векторов

int A[MAX_SIZE][MAX_SIZE] = {};
int B[MAX_SIZE][MAX_SIZE] = {};
int C[MAX_SIZE][MAX_SIZE] = {};


// Функция для перемножения квадратных матриц
void multiply_matrices(int A[MAX_SIZE][MAX_SIZE], int B[MAX_SIZE][MAX_SIZE], int C[MAX_SIZE][MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i][j] = 0;  // Инициализируем элемент C[i][j]
            for (int k = 0; k < N; k++) {
                C[i][j] += A[i][k] * B[k][j];  // Умножаем и суммируем
            }
        }
    }
}

// Функция для умножения матрицы на вектор
void multiply_matrix_vector(int A[MAX_SIZE][MAX_SIZE], int B[MAX_SIZE], int C[MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        C[i] = 0;  // Инициализируем элемент C[i]
        for (int j = 0; j < N; j++) {
            C[i] += A[i][j] * B[j];  // Умножаем и суммируем
        }
    }
}

// Функция для вывода матрицы
void print_matrix(int A[MAX_SIZE][MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", A[i][j]);
        }
        printf("\n");
    }
}

// Функция для вывода вектора
void print_vector(int B[MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        printf("%d ", B[i]);
    }
    printf("\n");
}

int main (int argc, char *argv[]) {
//int main(){

    char *a = argv[1];
    //printf("a = %s\n", a);

    int N = atoi(a);
    printf("N = %d\n", N);

    // Инициализация первой матрицы
    // printf("Инициализация первой матрицу A (%d x %d)...\n", N, N);
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i][j] = i*N + j;
        }
    }

    // Инициализация второй матрицы
    // printf("Инициализация второй матрицы B (%d x %d)...\n", N, N);
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            B[i][j] = i*N +j;
        }
    }

    // Перемножение матриц
    printf("Перемножение матриц C = A * B ...\n");
    multiply_matrices(A, B, C, N);
    //printf("Результат перемножения матриц C = A * B:\n");
    //print_matrix(C, N);

    // Умножение матрицы A на вектор
    int vector[MAX_SIZE];
    int result_vector[MAX_SIZE];

    //printf("Инициализация ветора размером %d ...\n", N);
    for (int i = 0; i < N; i++) {
        vector[i] = i;
    }

    printf("Умножение матрицы A на вектор ...\n");
    multiply_matrix_vector(A, vector, result_vector, N);
    //printf("Результат умножения матрицы A на вектор:\n");
    //print_vector(result_vector, N);

    printf("Завершил работу\n");
    return 0;
}


Overwriting matrix_operations.c


In [46]:
!gcc matrix_operations.c -o matrix_operations

In [47]:
%%timeit -r 1 -n 1
!./matrix_operations 512

N = 512
Перемножение матриц C = A * B ...
Умножение матрицы A на вектор ...
Завершил работу
1.51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


2. Разделить программу на несколько модулей и провести сборку через статическую линковку (25%)

In [ ]:
# можно используя знак ! запустить команду сборки прямо отсюда (так будет видно, что пункт 2 выполнен

In [75]:
%%writefile matrix_operations.h
#define MAX_SIZE 8192 // Максимальный размер матриц и векторов

void multiply_matrices(int A[MAX_SIZE][MAX_SIZE], int B[MAX_SIZE][MAX_SIZE], int C[MAX_SIZE][MAX_SIZE], int N);
void multiply_matrix_vector(int A[MAX_SIZE][MAX_SIZE], int B[MAX_SIZE], int C[MAX_SIZE], int N);

extern int A[MAX_SIZE][MAX_SIZE];
extern int B[MAX_SIZE][MAX_SIZE];
extern int C[MAX_SIZE][MAX_SIZE];

Overwriting matrix_operations.h


In [74]:
%%writefile matrix_operations_lib.c

#include <stdio.h>
#include <stdlib.h>

#include "matrix_operations.h"

int A[MAX_SIZE][MAX_SIZE] = {};
int B[MAX_SIZE][MAX_SIZE] = {};
int C[MAX_SIZE][MAX_SIZE] = {};

// Функция для перемножения квадратных матриц
void multiply_matrices(int A[MAX_SIZE][MAX_SIZE], int B[MAX_SIZE][MAX_SIZE], int C[MAX_SIZE][MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i][j] = 0;  // Инициализируем элемент C[i][j]
            for (int k = 0; k < N; k++) {
                C[i][j] += A[i][k] * B[k][j];  // Умножаем и суммируем
            }
        }
    }
}

// Функция для умножения матрицы на вектор
void multiply_matrix_vector(int A[MAX_SIZE][MAX_SIZE], int B[MAX_SIZE], int C[MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        C[i] = 0;  // Инициализируем элемент C[i]
        for (int j = 0; j < N; j++) {
            C[i] += A[i][j] * B[j];  // Умножаем и суммируем
        }
    }
}

// Функция для вывода матрицы
void print_matrix(int A[MAX_SIZE][MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", A[i][j]);
        }
        printf("\n");
    }
}

// Функция для вывода вектора
void print_vector(int B[MAX_SIZE], int N) {
    for (int i = 0; i < N; i++) {
        printf("%d ", B[i]);
    }
    printf("\n");
}



Overwriting matrix_operations_lib.c


In [71]:
%%writefile matrix_operations_main.c

#include <stdio.h>
#include <stdlib.h>

#include "matrix_operations.h"

int main (int argc, char *argv[]) {
//int main(){

    char *a = argv[1];
    //printf("a = %s\n", a);

    int N = atoi(a);
    printf("N = %d\n", N);

    // Инициализация первой матрицы
    // printf("Инициализация первой матрицу A (%d x %d)...\n", N, N);
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i][j] = i*N + j;
        }
    }

    // Инициализация второй матрицы
    // printf("Инициализация второй матрицы B (%d x %d)...\n", N, N);
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            B[i][j] = i*N +j;
        }
    }

    // Перемножение матриц
    printf("Перемножение матриц C = A * B ...\n");
    multiply_matrices(A, B, C, N);
    //printf("Результат перемножения матриц C = A * B:\n");
    //print_matrix(C, N);

    // Умножение матрицы A на вектор
    int vector[MAX_SIZE];
    int result_vector[MAX_SIZE];

    //printf("Инициализация ветора размером %d ...\n", N);
    for (int i = 0; i < N; i++) {
        vector[i] = i;
    }

    printf("Умножение матрицы A на вектор ...\n");
    multiply_matrix_vector(A, vector, result_vector, N);
    //printf("Результат умножения матрицы A на вектор:\n");
    //print_vector(result_vector, N);

    printf("Завершил работу\n");
    return 0;
}


Overwriting matrix_operations_main.c


In [76]:
!gcc -o matrix_operations_main matrix_operations_lib.c matrix_operations_main.c

In [79]:
 # проверяем работу
 !./matrix_operations_main 10

N = 10
Перемножение матриц C = A * B ...
Умножение матрицы A на вектор ...
Завершил работу


3. Подготовьте две сборки с флагами -g и -O3 и измерьте времена выполнения операций с N = 512, 1024, . . ., 4096 (20%)

In [ ]:
# замеры можно сделать используя %%time, %%timeit
# если комфортней работать с терминалом, то можно приложить
# к домашнему заданию скрин с записанными временами расчетов

4. Выполните вызов процедуры из Python через Ctypes/Cython/PyBind11 и измерьте времена (40%)

In [ ]:
# можно это сделать здесь

БОНУСЫ:

1. Дополнительные баллы за использование функций BLAS/cBLAS/openBLAS;
2. Дополнительные баллы за вызов теста LINPACK на вашем компьютере;
3. Супербонус: реализовать метод Штрассена перемножения квадратных матриц